In [1]:
import os
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')
OPENAI_API_KEY=os.getenv('OPENAI_API_KEY')

In [5]:
# loader
# use the data from documents to create vector store
from langchain.document_loaders import DirectoryLoader, TextLoader

loader = DirectoryLoader('./data', glob="**/*.txt", loader_cls=TextLoader, show_progress=True)

data = loader.load()

100%|██████████| 9/9 [00:00<00:00, 8996.36it/s]


In [6]:
# chunking the data to store in the database
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100
)

documents = text_splitter.split_documents(data)

In [7]:
# embeddings from huggingface
# from langchain.embeddings import HuggingFaceInstructEmbeddings
# embeddings = HuggingFaceInstructEmbeddings(model_name='hkunlp/instructor-large')

C:\Users\niran\anaconda3\envs\pytorch_learning\lib\site-packages\InstructorEmbedding\instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange


load INSTRUCTOR_Transformer
max_seq_length  512


In [19]:
# embeddings from openai
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

In [20]:
# loading vectors into VectorDB
from langchain.vectorstores.faiss import FAISS

vectorstore = FAISS.from_documents(documents, embeddings)
vectorstore.save_local("vectorstore")

In [29]:
# load the vectorstore
vectorstore = FAISS.load_local("vectorstore", embeddings)

In [30]:
# Prompt to answer question
from langchain.prompts import PromptTemplate

prompt_template = """You are a helpful assistant for our restaurant that answers the queries of the customer

{context}
Question: {question}
Answer here: """

PROMPT = PromptTemplate(
    template = prompt_template, input_variables=["context","question"]
)

In [31]:
# memory to store chat history
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True, output_key="answer")

In [32]:
from langchain.llms import OpenAI, HuggingFaceHub
from langchain.chains import ConversationalRetrievalChain

chain_type = {"prompt": PROMPT}
# to use open ai, uncomment it
llm = OpenAI(openai_api_key=OPENAI_API_KEY)

# to use models from huggingface, uncomment this
# llm = HuggingFaceHub(repo_id="google/flan-t5-xxl", model_kwargs={"temperature":0.5, "max_length":512})

qa_chain = ConversationalRetrievalChain.from_llm(
    llm,
    memory=memory,
    retriever=vectorstore.as_retriever(),
    combine_docs_chain_kwargs=chain_type
)

In [33]:
query="What are your pricing"


In [34]:
qa_chain({"question": query})

{'question': 'What are your pricing',
 'chat_history': [HumanMessage(content='What are your pricing'),
  AIMessage(content='Our pricing varies depending on the menu item. Our starters range from $7-$9, main courses from $16-$26, desserts around $7-$9, and beverages starting at $2. Please refer to our menu for specific item prices.')],
 'answer': 'Our pricing varies depending on the menu item. Our starters range from $7-$9, main courses from $16-$26, desserts around $7-$9, and beverages starting at $2. Please refer to our menu for specific item prices.'}

In [35]:
qa_chain({"question": "Why should I come here"})


{'question': 'Why should I come here',
 'chat_history': [HumanMessage(content='What are your pricing'),
  AIMessage(content='Our pricing varies depending on the menu item. Our starters range from $7-$9, main courses from $16-$26, desserts around $7-$9, and beverages starting at $2. Please refer to our menu for specific item prices.'),
  HumanMessage(content='Why should I come here'),
  AIMessage(content=" Gourmet Bistro is a great choice for dining because of our delicious and diverse menu, cozy ambiance, and convenient location. We also pride ourselves on exceptional service and have received numerous awards and recognitions for our culinary offerings. Whether you're looking for a casual lunch, a romantic dinner, or a family gathering, Gourmet Bistro has something for everyone.")],
 'answer': " Gourmet Bistro is a great choice for dining because of our delicious and diverse menu, cozy ambiance, and convenient location. We also pride ourselves on exceptional service and have received n